<a href="https://colab.research.google.com/github/Blessedtomato/TotoPrediction/blob/main/Toto5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#Import the relevant libraries and packages

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Xisiqomelir/xisiqo_datadump/main/ToTo.csv")
#Load the required dataset using pandas’s read CSV function

In [3]:
df.head()
#Taking a closer look at the data took help of “head()”function

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number,From Last,...,Division 3 Winners,Division 3 Prize,Division 4 Winners,Division 4 Prize,Division 5 Winners,Division 5 Prize,Division 6 Winners,Division 6 Prize,Division 7 Winners,Division 7 Prize
0,3884,2023-07-06,13,16,28,34,38,40,47,"34,40",...,147.0,1666.0,263.0,508.0,6591.0,50.0,7890.0,25.0,121282.0,10.0
1,3883,2023-07-03,14,30,34,40,45,49,12,30,...,85.0,2223.0,235.0,439.0,4664.0,50.0,6423.0,25.0,87982.0,10.0
2,3882,2023-06-29,5,15,16,30,37,46,49,5,...,515.0,1661.0,1208.0,387.0,25404.0,50.0,33315.0,25.0,475093.0,10.0
3,3881,2023-06-26,4,5,17,28,39,44,31,28,...,237.0,1741.0,531.0,424.0,12396.0,50.0,16290.0,25.0,232715.0,10.0
4,3880,2023-06-22,3,9,28,32,37,46,7,"28,32,46",...,112.0,2163.0,358.0,369.0,6596.0,50.0,11867.0,25.0,125106.0,10.0


In [4]:
df.info()
#Check attributes names and datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Draw                1544 non-null   int64  
 1   Date                1544 non-null   object 
 2   Winning Number 1    1544 non-null   int64  
 3   2                   1544 non-null   int64  
 4   3                   1544 non-null   int64  
 5   4                   1544 non-null   int64  
 6   5                   1544 non-null   int64  
 7   6                   1544 non-null   int64  
 8   Additional Number   1544 non-null   int64  
 9   From Last           915 non-null    object 
 10  Same As Day         208 non-null    float64
 11  Odd                 1544 non-null   int64  
 12  Even                1544 non-null   int64  
 13   1-10               1544 non-null   int64  
 14   11-20              1544 non-null   int64  
 15   21-30              1544 non-null   int64  
 16   31-40

In [5]:
df.Date = pd.to_datetime(df.Date)
#Converting Date column to datetime type

In [6]:
df.drop(['From Last', 'Same As Day', 'Odd', 'Even', ' 1-10', ' 11-20', ' 21-30', ' 31-40', ' 41-50', 'Division 1 Winners', 'Division 1 Prize', 'Division 2 Winners', 'Division 2 Prize', 'Division 3 Winners', 'Division 3 Prize', 'Division 4 Winners', 'Division 4 Prize', 'Division 5 Winners', 'Division 5 Prize', 'Division 6 Winners', 'Division 6 Prize', 'Division 7 Winners', 'Division 7 Prize'], axis=1, inplace=True)
df.head()
#Dropping unwanted features

,Draw,Date,Winning Number 1,2,3,4,5,6,Additional Number
0,3884,2023-07-06,13,16,28,34,38,40,47
1,3883,2023-07-03,14,30,34,40,45,49,12
2,3882,2023-06-29,5,15,16,30,37,46,49
3,3881,2023-06-26,4,5,17,28,39,44,31
4,3880,2023-06-22,3,9,28,32,37,46,7


In [7]:
df.drop(['Date', 'Draw'], axis=1, inplace=True)
df.head()

,Winning Number 1,2,3,4,5,6,Additional Number
0,13,16,28,34,38,40,47
1,14,30,34,40,45,49,12
2,5,15,16,30,37,46,49
3,4,5,17,28,39,44,31
4,3,9,28,32,37,46,7


In [8]:
df.tail()

,Winning Number 1,2,3,4,5,6,Additional Number
1539,5,10,25,27,36,39,40
1540,5,12,17,18,38,40,22
1541,22,32,41,42,43,44,31
1542,2,18,19,21,39,45,36
1543,6,11,14,15,28,45,35


In [9]:
print(df.shape)
#890 records, 7 fields

(1544, 7)


In [10]:
df.describe()

,Winning Number 1,2,3,4,5,6,Additional Number
count,1544.000000,1544.000000,1544.000000,1544.000000,1544.000000,1544.000000,1544.000000
mean,6.825777,13.696891,20.608808,27.627591,34.389896,41.380829,24.078368
std,5.431965,7.067866,7.762587,7.800446,7.245231,5.854914,13.816207
min,1.000000,2.000000,3.000000,7.000000,12.000000,13.000000,1.000000
25%,3.000000,8.000000,15.000000,22.000000,29.000000,38.000000,12.000000
50%,5.000000,13.000000,20.000000,28.000000,35.000000,43.000000,24.000000
75%,9.000000,18.000000,26.000000,33.000000,40.000000,46.000000,36.000000
max,35.000000,38.000000,43.000000,47.000000,48.000000,49.000000,49.000000


In [11]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [12]:
transformed_df.head()
#Scaled data

,0,1,2,3,4,5,6
0,1.137015,0.325962,0.952464,0.817193,0.498434,-0.235917,1.659577
1,1.321170,2.307400,1.725653,1.586629,1.464900,1.301751,-0.874501
2,-0.336226,0.184431,-0.593913,0.304236,0.360368,0.789195,1.804381
3,-0.520381,-1.230882,-0.465048,0.047758,0.636501,0.447491,0.501142
4,-0.704536,-0.664757,0.952464,0.560715,0.360368,0.789195,-1.236512


In [13]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

1544

In [14]:
# Amount of games we need to take into consideration for prediction
window_length = 50
window_length

50

In [15]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

7

In [16]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [17]:
X.shape

(1494, 50, 7)

In [18]:
y.shape

(1494, 7)

In [19]:
X[0]

array([[ 1.13701454,  0.3259619 ,  0.95246422,  0.81719348,  0.4984345 ,
        -0.23591744,  1.65957697],
       [ 1.32116963,  2.3073996 ,  1.72565283,  1.58662944,  1.46490024,
         1.30175083, -0.87450061],
       [-0.33622616,  0.18443064, -0.59391302,  0.30423618,  0.36036796,
         0.78919474,  1.80438141],
       [-0.52038125, -1.23088201, -0.46504825,  0.04775752,  0.63650103,
         0.44749068,  0.50114151],
       [-0.70453634, -0.66475695,  0.95246422,  0.56071483,  0.36036796,
         0.78919474, -1.23651169],
       [-0.88869142,  1.59974328,  0.95246422,  0.30423618, -0.32996471,
         0.78919474,  1.73197919],
       [-0.15207107,  0.18443064, -0.46504825,  1.8431081 ,  1.60296677,
         1.30175083, -0.29528287],
       [-0.88869142,  1.17514949,  0.95246422,  0.68895416, -0.05383164,
        -0.91932556, -1.23651169],
       [-0.15207107,  1.03361822,  1.33905852,  0.56071483,  0.08423489,
         1.1308988 , -0.36768509],
       [-0.33622616,  0.3259

In [20]:
y[0]

array([-0.88869142, -1.08935074, -1.62483117,  0.04775752, -0.46803124,
       -0.74847353,  1.73197919])

In [21]:
X[1]

array([[ 1.32116963,  2.3073996 ,  1.72565283,  1.58662944,  1.46490024,
         1.30175083, -0.87450061],
       [-0.33622616,  0.18443064, -0.59391302,  0.30423618,  0.36036796,
         0.78919474,  1.80438141],
       [-0.52038125, -1.23088201, -0.46504825,  0.04775752,  0.63650103,
         0.44749068,  0.50114151],
       [-0.70453634, -0.66475695,  0.95246422,  0.56071483,  0.36036796,
         0.78919474, -1.23651169],
       [-0.88869142,  1.59974328,  0.95246422,  0.30423618, -0.32996471,
         0.78919474,  1.73197919],
       [-0.15207107,  0.18443064, -0.46504825,  1.8431081 ,  1.60296677,
         1.30175083, -0.29528287],
       [-0.88869142,  1.17514949,  0.95246422,  0.68895416, -0.05383164,
        -0.91932556, -1.23651169],
       [-0.15207107,  1.03361822,  1.33905852,  0.56071483,  0.08423489,
         1.1308988 , -0.36768509],
       [-0.33622616,  0.3259619 ,  0.69473468,  0.04775752, -0.32996471,
        -1.26102962, -0.00567401],
       [ 0.03208401,  0.1844

In [22]:
y[1]

array([-1.07284651, -1.51394453, -0.98050733, -1.61935373, -1.71063006,
        0.27663865,  1.58717476])

In [23]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.15, random_state = 31)

In [24]:
model = Sequential()
#Initialise RNN

In [25]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
#Add the input layer and the LSTM layer

In [26]:
model.add(Dropout(0.2))
#Add a first Dropout layer in order to reduce overfitting

In [27]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
#Add a second LSTM layer

In [28]:
model.add(Dropout(0.2))
#Add a second Dropout layer

In [29]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
#Add a third LSTM layer

In [30]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))
#Add a fourth LSTM layer

In [31]:
model.add(Dense(59))
#Dense layer

In [32]:
model.add(Dense(number_of_features))
#Add the last output layer

In [33]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])
#Compiling the RNN

In [35]:
history=model.fit(X, y, batch_size=400, epochs=400, verbose=2) #validation_data=(X_test, y_test))

Epoch 1/400
4/4 - 55s - loss: 0.9674 - accuracy: 0.2068 - 55s/epoch - 14s/step
Epoch 2/400
4/4 - 48s - loss: 0.9672 - accuracy: 0.2088 - 48s/epoch - 12s/step
Epoch 3/400
4/4 - 48s - loss: 0.9662 - accuracy: 0.2122 - 48s/epoch - 12s/step
Epoch 4/400
4/4 - 45s - loss: 0.9648 - accuracy: 0.2041 - 45s/epoch - 11s/step
Epoch 5/400
4/4 - 47s - loss: 0.9637 - accuracy: 0.2021 - 47s/epoch - 12s/step
Epoch 6/400
4/4 - 46s - loss: 0.9630 - accuracy: 0.2015 - 46s/epoch - 11s/step
Epoch 7/400
4/4 - 47s - loss: 0.9628 - accuracy: 0.1968 - 47s/epoch - 12s/step
Epoch 8/400
4/4 - 46s - loss: 0.9612 - accuracy: 0.2048 - 46s/epoch - 11s/step
Epoch 9/400
4/4 - 47s - loss: 0.9607 - accuracy: 0.2075 - 47s/epoch - 12s/step
Epoch 10/400
4/4 - 47s - loss: 0.9599 - accuracy: 0.2068 - 47s/epoch - 12s/step
Epoch 11/400
4/4 - 46s - loss: 0.9594 - accuracy: 0.2108 - 46s/epoch - 11s/step
Epoch 12/400
4/4 - 47s - loss: 0.9591 - accuracy: 0.2115 - 47s/epoch - 12s/step
Epoch 13/400
4/4 - 46s - loss: 0.9585 - accuracy:

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 50, 480)          476160    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 50, 480)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 50, 480)          1384320   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 50, 480)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 50, 480)          1384320   
 nal)                                                            
                                                        

In [61]:
to_predict = df.head(8)
to_predict

,Winning Number 1,2,3,4,5,6,Additional Number
0,13,16,28,34,38,40,47
1,14,30,34,40,45,49,12
2,5,15,16,30,37,46,49
3,4,5,17,28,39,44,31
4,3,9,28,32,37,46,7
5,2,25,28,30,32,46,48
6,6,15,17,42,46,49,20
7,2,22,28,33,34,36,7


In [62]:
to_predict = np.array(to_predict)
to_predict

array([[13, 16, 28, 34, 38, 40, 47],
       [14, 30, 34, 40, 45, 49, 12],
       [ 5, 15, 16, 30, 37, 46, 49],
       [ 4,  5, 17, 28, 39, 44, 31],
       [ 3,  9, 28, 32, 37, 46,  7],
       [ 2, 25, 28, 30, 32, 46, 48],
       [ 6, 15, 17, 42, 46, 49, 20],
       [ 2, 22, 28, 33, 34, 36,  7]])

In [63]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

array([[ 1.13701454,  0.3259619 ,  0.95246422,  0.81719348,  0.4984345 ,
        -0.23591744,  1.65957697],
       [ 1.32116963,  2.3073996 ,  1.72565283,  1.58662944,  1.46490024,
         1.30175083, -0.87450061],
       [-0.33622616,  0.18443064, -0.59391302,  0.30423618,  0.36036796,
         0.78919474,  1.80438141],
       [-0.52038125, -1.23088201, -0.46504825,  0.04775752,  0.63650103,
         0.44749068,  0.50114151],
       [-0.70453634, -0.66475695,  0.95246422,  0.56071483,  0.36036796,
         0.78919474, -1.23651169],
       [-0.88869142,  1.59974328,  0.95246422,  0.30423618, -0.32996471,
         0.78919474,  1.73197919],
       [-0.15207107,  0.18443064, -0.46504825,  1.8431081 ,  1.60296677,
         1.30175083, -0.29528287],
       [-0.88869142,  1.17514949,  0.95246422,  0.68895416, -0.05383164,
        -0.91932556, -1.23651169]])

In [64]:
y_pred = model.predict(np.array([scaled_to_predict]))
print('The predicted numbers in the last lottery game are:', scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 33ms/step
The predicted numbers in the last lottery game are: [10 14 21 24 31 39 33]
